In [ ]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
ash_excitation = []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity


#Hamiltonian 
symbols = ["H","H","H","H"]
r_bohr = 5.0*1.8897259886
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf")

#print('The original hamiltonian is', H)
electrons = 4
orbitals = 8
singles, doubles = qml.qchem.excitations(electrons, orbitals,fermionic=True)
print('Singles are',singles)
print('Doubles are',doubles)
hf_state = qchem.hf_state(electrons, qubits)
print(f"Total number of excitations = {len(singles) + len(doubles)}")

#Calculation of HF state
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def circuit(hf_state, electrons, qubits, H):
    # Prepare the Hartree-Fock state
    print('Updated hf_state is', hf_state)
    qml.BasisState(hf_state, wires=range(qubits))

    return qml.expval(H)
print('HF state is', circuit(hf_state, electrons, qubits, H))

#Putting all the functions here
#1.1091571486954503

In [ ]:
# Commutator calculation 
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def commutator_0(H,w, k):
    qml.BasisState(k, wires=range(qubits))
    res = qml.commutator(H, w)
    #res = 2*(qml.prod(H, w)) - (qml.prod(w, H))
    return qml.expval(res)

# Commutator calculation 
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def commutator_1(H,w, k):
    qml.StatePrep(k, wires=range(qubits))
    res = qml.commutator(H, w)
    #res = 2*(qml.prod(H, w)) - (qml.prod(w, H))
    return qml.expval(res)

# Energy calculation 
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def ash(params, ash_excitation, hf_state, qubits, H):
    qml.BasisState(hf_state, wires=range(qubits))
    for i, excitation in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
        elif len(ash_excitation[i]) == 2:
            print('Single Exc going in is', ash_excitation[i])
            qml.FermionicSingleExcitation(weight=params[i], wires=ash_excitation[i])
            #qml.SingleExcitation(params[i], wires=ash_excitation[i])
    return qml.expval(H)

# Calculation of New state 
#print('Now we are going to calculate the new_state')
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def new_state(hf_state, ash_excitation, params, fparams):
    qml.BasisState(hf_state, wires=range(qubits))
    print('All params in state prep are', fparams)
    for i, excitations in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            print('Exc. dealing right now is', ash_excitation[i])
            print('The params that are going in', fparams[i])
            #print('The wires1 are', ash_excitation[i][2:][::-1])
            qml.DoubleExcitation(fparams[i], wires = ash_excitation[i])
            #qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
        elif len(ash_excitation[i]) == 2:
            print('Single Exc. dealing right now is', ash_excitation[i])
            print('Single exc params that are going in', fparams[i])
            #qml.FermionicSingleExcitation(weight=params[i], wires=ash_excitation[i])
            qml.SingleExcitation(fparams[i], wires=ash_excitation[i])

    return qml.state()

In [ ]:

electrons = 4
orbitals = 8
singles, doubles = qml.qchem.excitations(electrons, orbitals, fermionic=True)
optimizer = qml.GradientDescentOptimizer(stepsize=0.5)

print('The Hamiltonian is ', H)
adit = 6
fparams = []
excitations= []
operator_pool = (singles) + (doubles)
states = [hf_state]


for j in range(adit):
    print('The adapt iteration now is', j)
    max_value = float('-inf')
    max_operator = None
    # Start with the most recent state (last state in the list)
    k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state

    for i in operator_pool:
        print('The current excitation operator is', i)
        w = qml.fermi.jordan_wigner(i)
        if np.array_equal(k, hf_state):
            print('Print, if this is activated - HF state')
            current_value = abs(commutator_0(H, w, k))
        else:
            current_value = abs(commutator_1(H, w, k))
        print(f'The expectation value of {i} is', current_value)

        if current_value > max_value:
            max_value = current_value
            max_operator = i

    print(f"The highest operator value is {max_value} for operator {max_operator}")

    # Convert operator to excitations and append to ash_excitation
    indices_str = re.findall(r'\d+', str(max_operator))
    excitations = [int(index) for index in indices_str]
    print('Highest gradient excitation is', excitations)
    ash_excitation.append(excitations)
    print('The current status of ash_excitation is', ash_excitation)
    print('Moving towards parameters')
    params = np.zeros(len(ash_excitation), requires_grad=True)
    print('The length of parameters is', len(params))
    
    # Cost function definition
    cost_fn = qml.QNode(ash, dev, interface="autograd", diff_method="adjoint")
    print('Going to do energy calculation')
    for n in range(150):
        print(f'Each step, the iteration is {n} and the parameter is {params}')
        params, energy = optimizer.step_and_cost(cost_fn, params, ash_excitation=ash_excitation, hf_state=hf_state, qubits=qubits, H=H)
        if n % 5 == 0:
            print(f"step = {n}, E = {energy:.8f} Ha")
    fparams.append(params)
    print('Updated params are', fparams)
    print('Updated excitation are', ash_excitation)
    
    # New state generation
    ostate = new_state(hf_state, ash_excitation, params, fparams)
    print(qml.draw(new_state, level="device", max_length=100)(hf_state,ash_excitation,params,fparams))
    print('The updated state is', ostate)
    # Append the new state to the states list
    states.append(ostate)  


## LiH @ 5.0A

In [2]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
ash_excitation = []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity


#Hamiltonian 
symbols = ["H","H","H","H"]
#symbols = ["Li", "H"]
r_bohr = 3.0*1.8897259886  #5.0 Angstrom in Bohr 
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
#coordinates = np.array([[0.0,0.0, 0*r_bohr], [0.0, 0.0, 1*r_bohr]])
# Calculation of the Hamiltonian
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf")


electrons = 4
orbitals = 8
# Calculation of the excitation operators
singles, doubles = qml.qchem.excitations(electrons, orbitals,fermionic=False)
print('Singles are',singles)
print('Doubles are',doubles)
hf_state = qchem.hf_state(electrons, qubits)
print(f"Total number of excitations = {len(singles) + len(doubles)}")

#Calculation of HF state
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def circuit(hf_state, electrons, qubits, H):
    # Prepare the Hartree-Fock state
    print('Updated hf_state is', hf_state)  #HF state : [1,1,1,1,0,0,0,0] - [alpha, beta, alpha, beta, alpha, beta, alpha, beta]
    qml.BasisState(hf_state, wires=range(qubits))

    return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
print('HF state is', circuit(hf_state, electrons, qubits, H))

print('The original hamiltonian is', H)
#Putting all the functions here
#1.1091571486954503

Singles are [[0, 4], [0, 6], [1, 5], [1, 7], [2, 4], [2, 6], [3, 5], [3, 7]]
Doubles are [[0, 1, 4, 5], [0, 1, 4, 7], [0, 1, 5, 6], [0, 1, 6, 7], [0, 2, 4, 6], [0, 3, 4, 5], [0, 3, 4, 7], [0, 3, 5, 6], [0, 3, 6, 7], [1, 2, 4, 5], [1, 2, 4, 7], [1, 2, 5, 6], [1, 2, 6, 7], [1, 3, 5, 7], [2, 3, 4, 5], [2, 3, 4, 7], [2, 3, 5, 6], [2, 3, 6, 7]]
Total number of excitations = 26


ImportError: Pre-compiled binaries for lightning.qubit are not available. To manually compile from source, follow the instructions at https://docs.pennylane.ai/projects/lightning/en/stable/dev/installation.html.

In [ ]:
# Commutator calculation for HF state
dev = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev)
def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
    qml.BasisState(k, wires=range(qubits))
    res = qml.commutator(H, w)   #Calculating the commutator
    #res = (qml.prod(H, w)) - (qml.prod(w, H))
    return qml.expval(res)

# Commutator calculation for other states except HF state
dev = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev)
def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
    qml.StatePrep(k, wires=range(qubits))
    res = qml.commutator(H, w) #Calculating the commutator
    return qml.expval(res)

# Energy calculation 
dev = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev)
def ash(params, ash_excitation, hf_state, qubits, H):
    #qml.BasisState(hf_state, wires=range(qubits))
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
    for i, excitation in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            #qml.DoubleExcitation(params[i], wires = ash_excitation[i])  #Applying the double excitation and their parameters
            qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
        elif len(ash_excitation[i]) == 2:
            print('Single Exc going in is', ash_excitation[i])
            #qml.SingleExcitation(params[i], wires = ash_excitation[i]) #Applying the single excitation and their parameters
            qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))

    return qml.expval(H)  #Calculating the expectation value of the Hamiltonian

# Calculation of New state, same as the above function but with the state return

dev = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev)
def new_state(hf_state, ash_excitation, fparams, params):
    #qml.BasisState(hf_state, wires=range(qubits))
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
    for i, excitations in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            print('Exc. dealing right now is', ash_excitation[i])
            print('The params that are going in', params[i])
            #qml.DoubleExcitation(params[i], wires = ash_excitation[i])
            qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])

        elif len(ash_excitation[i]) == 2:
            print('Single Exc. dealing right now is', ash_excitation[i])
            print('Single exc params that are going in', params[i])
            #qml.SingleExcitation(params[i], wires = ash_excitation[i])
            qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))

            

    return qml.state()

In [ ]:

electrons = 4
orbitals = 8
singles, doubles = qml.qchem.excitations(electrons, orbitals, fermionic=True)
optimizer = qml.GradientDescentOptimizer(stepsize=0.5)  #Optimizer

print('The Hamiltonian is ', H)
adit = 10
fparams = []
excitations= []
operator_pool = (singles) + (doubles)  #Operator pool - Singles and Doubles
states = [hf_state]


for j in range(adit):
    print('The adapt iteration now is', j)  #Adapt iteration
    max_value = float('-inf')
    max_operator = None
    # Start with the most recent state (last state in the list)
    k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state

    for i in operator_pool:
        print('The current excitation operator is', i)   #Current excitation operator - fermionic one
        w = qml.fermi.jordan_wigner(i)  #JW transformation
        if np.array_equal(k, hf_state): # If the current state is the HF state
            print('Print, if this is activated - HF state')
            current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
        else:
            current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
        print(f'The expectation value of {i} is', current_value)

        if current_value > max_value:
            max_value = current_value
            max_operator = i

    print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

    # Convert operator to excitations and append to ash_excitation
    indices_str = re.findall(r'\d+', str(max_operator))
    excitations = [int(index) for index in indices_str]
    print('Highest gradient excitation is', excitations)
    ash_excitation.append(excitations) #Appending the excitations to the ash_excitation
    print('The current status of ash_excitation is', ash_excitation)
    print('Moving towards parameters')
    params = np.zeros(len(ash_excitation), requires_grad=True)  #Parameters initialization
    print('The length of parameters is', len(params))
    
    # Cost function definition
    cost_fn = qml.QNode(ash, dev, interface="autograd", diff_method="backprop") #Cost function definition, gradient calculated automatically and method is backprop
    print('Going to do energy calculation')
    tolerance = 1e-11  # Set your desired tolerance
    previous_energy = float('inf') 
    for n in range(1500):
        print(f'Each step, the iteration is {n} and the parameter is {params}')
        params, energy = optimizer.step_and_cost(cost_fn, params, ash_excitation=ash_excitation, hf_state=hf_state, qubits=qubits, H=H)
        if n % 5 == 0:
            print(f"step = {n}, E = {energy:.8f} Ha")
            # Check for convergence based on energy difference
        if abs(energy - previous_energy) < tolerance:
            print(f"Converged after {n} iterations.")
            break

        previous_energy = energy 
    fparams.append(params)
    print('Updated params are', params)
    print('Updated excitation are', ash_excitation)
    
    # New state generation - With the updated parameters
    ostate = new_state(hf_state, ash_excitation, fparams, params)
    print(qml.draw(new_state, level="device", max_length=100)(hf_state,ash_excitation,fparams,params))
    print('The updated state is', ostate)
    # Append the new state to the states list
    states.append(ostate)  


In [ ]:
print(singles[0])
#print(doubles)

doubles = qml.qchem.excitations(electrons, orbitals, fermionic=False)
print(doubles)

doubles = [[0, 1, 4, 5]]
print(doubles)

singles = [[0,4]]

In [ ]:
singles = [[2,6]]
doubles = [[2,3,10,11]]
singles_wires, doubles_wires = qml.qchem.excitations_to_wires(singles, doubles)

print('Singles are', singles)
print(singles_wires)
print('Double wires are', doubles_wires)


In [ ]:
print(ash_excitation)
ash = [[2,6]]
wires=list(range(ash[0][0], ash[0][1] + 1))

print(wires)

In [ ]:
import pennylane as qml
from pennylane import numpy as np

# Define the molecule
symbols  = ['H', 'H', 'H']
geometry = np.array([[0.01076341,  0.04449877,  0.0],
                     [0.98729513,  1.63059094,  0.0],
                     [1.87262415, -0.00815842,  0.0]], requires_grad = False)
electrons = 2
charge = 1

# Build the electronic Hamiltonian
H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, charge=charge)

# Define the HF state
hf_state = qml.qchem.hf_state(electrons, qubits)

# Generate single and double excitations
singles, doubles = qml.qchem.excitations(electrons, qubits)
print(singles)
# Map excitations to the wires the UCCSD circuit will act on
s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
print(s_wires)
print(d_wires)

d_wires = [[0, 0], [0, 0]]
# Define the device
dev = qml.device("default.qubit", wires=qubits)

# Define the qnode
@qml.qnode(dev)
def circuit(params, wires, s_wires, d_wires, hf_state):
    qml.UCCSD(params, wires, s_wires,d_wires, hf_state)
    return qml.expval(H)

# Define the initial values of the circuit parameters
params = np.zeros(len(singles) + len(doubles))

# Define the optimizer
optimizer = qml.GradientDescentOptimizer(stepsize=0.5)

# Optimize the circuit parameters and compute the energy
for n in range(21):
    params, energy = optimizer.step_and_cost(circuit, params,
    wires=range(qubits), s_wires=s_wires, d_wires=d_wires, hf_state=hf_state)
    if n % 2 == 0:
        print("step = {:},  E = {:.8f} Ha".format(n, energy))